In [ ]:
import sys
sys.path.insert(0,'../libs')
import fourier as foo
import ecualizacion as equ
from matplotlib import pyplot as plt
import numpy as np
from scipy import fftpack
from scipy import misc
# import seaborn as sns
# sns.despine()
from numpy import linalg
from scipy.signal import convolve2d
import random

In [ ]:
def convolve(I, K):
    return convolve2d(I,K,mode="same")

In [ ]:
def show_img(IMG, fig_size=(18,10)):
    plt.figure(figsize=fig_size); plt.axis('off')
    plt.imshow(IMG ,cmap='gray')
    plt.show()
    
def show_img_s(IMG, titles=None, fig_size = (18,10)):
    img_qty = len(IMG)
    plt.figure(figsize=fig_size); 
    for i in range(img_qty):
        plt.subplot(1,img_qty, i+1)
        if titles != None:
            plt.title(titles[i])
        plt.imshow(IMG[i] ,cmap='gray'); plt.axis('off')
    plt.tight_layout()
    plt.show()

## Ejercicio 1

**freq_filter** toma como parámetros la imágen a filtrar, y una función de decisión. La misma, en pos de la *distancia* de una cierta amplitud para una frecuencia f al centro del espacio, decide si será removida o no.

In [ ]:
def freq_filter(img, CUTOFF_FUN):
    filtered_img = np.copy(img)
    m = len(img); n = len(img[0])
    distance = lambda i,j: np.sqrt(abs(i-m/2)**2 + abs(j-n/2)**2)
    for i in range(m):
        for j in range(n):
            if not CUTOFF_FUN(distance(i,j)):
                filtered_img[i][j] = 0
    return filtered_img

In [ ]:
img = misc.imread('lena.png')

**Nota:** Si en algun momento se reodernana los cuadrantes de la imagen en el dominio de las frecuencias, antes de ser convertido por la IFFT, los mismos deben tener el mismo orden; ya sea el original, o con el el (0,0) de las frecuencias centrado en la imagen. En el *code-block* de abajo se puede ver ejemplos de ciertos filtros. En cada uno, **l** y **h** hacen referencia a los valores de corte inferior y superior.

In [ ]:
BAND_PASS_FILTER = lambda l,h: lambda d: (d <= l) or (d> h)
LOW_PASS_FILTER = lambda l: lambda d: (d <= l)
HIGH_PASS_FILTER = lambda h: lambda d: (d > h)

In [ ]:
img_F = fftpack.fft2(img)
# Aca ya reordeno los valores en complejo de toda la imagen (parte real e imaginaria),
# de forma de operar directamente con ellos, sin preocuparme.
img_F = foo.fix_norm_plot_regions(img_F)
img_ABS = np.abs(img_F)
# funcion de corte
CUTOFF_FUNC = LOW_PASS_FILTER(50)
# aplico el filtro a la transformada
img_ABS = freq_filter(img_ABS, CUTOFF_FUNC)
# agregego los módulos filtrados con sus correspondientes ángulos
img_MOD = foo.assemble_complex(img_ABS, \
                               np.angle(img_F))
# anti transformo
img_deMOD = fftpack.ifft2(img_MOD)
show_img_s([img, foo.log_transform(img_ABS), np.abs(img_deMOD)],\
          ["original", "transformada filtrada", "antritransformada filtrada"])

La idea consiste en la siguiente conversión:

<p style="text-align: center;">

$\mathcal{F}(img)  \mathcal{F}(kernel) = \mathcal{F}(img_{filtered})$

$\mathcal{F}(img)^{-1}  \mathcal{F}(img)  \mathcal{F}(kernel) = \mathcal{F}(img)^{-1}  \mathcal{F}(img_{filtered})$

$\mathcal{F}(kernel) = \mathcal{F}(img)^{-1}  \mathcal{F}(img_{filtered})$

$\mathcal{F}(kernel) \xrightarrow[]{IDFT} kernel$

</p>
Pero el problema surge que para capturar una parte representativa del kernel, debo agrander su dimensión. Y esto solo sucede al realizar **lowPassFiltering**; en el caso contrario, no se logra el efecto deseado al aplicar el filtro resultante por convolución.

In [ ]:
# en esta operación realizo
"""
inversa(TRANSF(imagen original)) . TRANSF(imagen filtrada)
reordeno las regiones
antiTRANSFORMO
tomo muestra del kernel
"""
kernel_size = 100
KERN_F = foo.fix_norm_plot_regions(np.dot(linalg.inv(img_F),img_MOD))
KERN = np.abs(fftpack.ifft2(KERN_F[0:kernel_size,0:kernel_size]))
show_img_s([foo.log_transform(np.abs(KERN_F)), KERN, convolve(img, KERN)],\
           ["transformada del kernel obtenido", "antritransformada del kernel",\
            "convolucion con kernel obetinido"])

In [ ]:
filtered_img = np.dot(img_F, foo.fix_norm_plot_regions(KERN_F))
# print(filtered_img.shape)
show_img_s([foo.log_transform(np.abs(filtered_img)),\
            (np.abs(fftpack.ifft2(filtered_img)))])

### Average Filter
Actúa como filtro pasa-bajos. El mismo toma la media de una vecindad de $s^2$.

In [ ]:
LOW_PASS_KERNEL = lambda s: np.ones((s,s))*(1/float(s))

In [ ]:
show_img(convolve(img, LOW_PASS_KERNEL(5)))

## Ejercicio 2

In [ ]:
def randon_mask(alto, ancho, lower=-5, upper=10):
    k = np.zeros((alto, ancho))
    for i in range(k.shape[0]):
        for j in range(k.shape[1]):
            k[i,j] = random.uniform(lower, upper)
    return k   

La función *randon_mask* toma como parámetros las dimensiones del kernel a aplicar ($alto \geq 0 \land ancho \geq 0$), y los límites de los valores que contendrá el kernel ($lower \leq k_{i,j} \leq upper$), los cuales se generan de manera aleatoria.

Resulta de interés notat que para kernels pequeños, se obtiene más variedad en el efecto causado por las diferentes máscaras, ya que las muestra de la imágen sobre la que operan es menor, por lo que el peso que tiene cada valor elegido aleatoriamente aumenta. Por otro lado, para kernels de mayor tamaño, la importancia de cada elemento se encuentra más distribuido entre el total.

In [ ]:
kernel = randon_mask(3,1)
print("Se generó un kernel aleatorio: ")
print(kernel)
show_img_s([img, convolve(img, kernel)],\
           ["original", "kernel aleatorio"], fig_size = (15,6))

## Ejercicio 3

### Máscaras para suavizado

In [ ]:
def gaussian_kern(dim, sigma):
    KERN = np.zeros((dim, dim))
    gauss_func = lambda x,y: (1/(2*np.pi*sigma))*np.exp([\
        -(x**2 + y**2)/(2*sigma**2)\
        ])
    for i in range(dim):
        for j in range(dim):
            KERN[i,j] = gauss_func(abs(i-dim/2),abs(j-dim/2))
    return KERN
# show_img(gaussian_kern(5, 1.5))
def mean_kern(dim):
    return np.ones((dim,dim))/float(dim**2)

En las imagenes siguientes se puede ver una imagen de lena original, una suavizada con un filtro de la media, y una con un filtro guassiano (con una desvío standar de 1,5). Ambos kernel son de 5x5 pixels.

In [ ]:
show_img_s([img, convolve(img, mean_kern(5)), convolve(img, gaussian_kern(5,1.5))],\
           ["original lena", "mean filter", "guassian filter (SIGMA=1.5)"])

### Unshark Masking
Cuando se hace referencia a UM, quiere decirse Unsharp Masking.

In [ ]:
blurry_img = misc.imread("blurry_forest_2.jpg")[:,:,0]

In [ ]:
def unsharp_masking(img, alpha, size = 3):
    ID = np.zeros((size,size)); ID[size/2,size/2] = 1
    KERN = np.array(ID - mean_kern(size))
    return img+alpha*convolve2d(img, KERN, mode="same")
#     return img + alpha*convolve2d(img, KERN, mode="same")
def naive_unsharp_masking(img, alpha, size=3):
    return img + alpha*(img - convolve(img, mean_kern(size)))


In [ ]:
# mean_filtered_img = convolve2d(img_2, np.ones((3,3))*(1/float(9)), mode="same")
show_img_s([blurry_img, unsharp_masking(blurry_img, .7, size=5), unsharp_masking(blurry_img, 1, size=5), unsharp_masking(blurry_img, 2, size=5)],\
           ["original", "UM ALPHA = 0,7","UM ALPHA = 1", "UM ALPHA = 2" ])

## Ejercicio 4

### Implementación de varios tipos de ruido
En todas las funciones de ruido, se asume que la imagen pasadas como parámetros poseen un tipo de datos **uint8**.

### Salt & Pepper noise
El parámetro que toma es la probailidad que en un pixel se produzca ruido impuslivo. La misma se distribuye de la siguiente forma:

Si es $x \sim U(0,1)$, luego si $x \leq p/2$, el ruido aplicado es pepper; si $x \geq 1-p/2$, se aplica salt; caso contrario, no hay ruido.

In [ ]:
# P&S noise también es conocido como ruido impulsivo
def pep_salt_noise(img, prob):
    IMG = np.copy(img)
    p_p = prob/2; p_s=1-prob/2
    for i in range(len(IMG)):
        for j in range(len(IMG[0])):
            lottery = np.random.uniform()
            if lottery < p_p:
                # pepper
                IMG[i,j] = 0
            elif p_s < lottery:
                #salt
                IMG[i,j] = 255                
            else:
                #none
                continue    
    return IMG

In [ ]:
show_img_s([pep_salt_noise(img, .1), pep_salt_noise(img, .05), pep_salt_noise(img, .025),\
            pep_salt_noise(img, .01)], ["p = 0,1", "p = 0,05", "p = 0,025", "p = 0,01"])

### Ruido Gaussiano aditivo

In [ ]:
def additive_gaussian_noise(img, mean, sd):
    IMG = np.copy(img)
    for i in range(len(IMG)):
        for j in range(len(IMG[0])):
            IMG[i,j] += int(random.gauss(mean, sd))
    return IMG

In [ ]:
sigma_range = [1,10,25,50]
contamined_imgs = [additive_gaussian_noise(img, 0,sigma) for sigma in sigma_range]
contamined_imgs_labels = ["sigma = " + str(sigma) for sigma in sigma_range]
print("------------ IMAGENES CONTAMINADAS ------------")
show_img_s(contamined_imgs, contamined_imgs_labels)
print("------------ FILTRO MEDIA ------------")
show_img_s([convolve(i, mean_kern(3)) for i in contamined_imgs])

In [ ]:
print("------------ UNSHARP_MASKSING ------------")
show_img_s([unsharp_masking(i, 1) for i in contamined_imgs])

### Ruido multiplicativo de Rayleigh

Én la función implementada, se tomo como parámetros las media y la variancia de la distribución del ruido (Rayleigh según diapositivas y González), y de ahí se calcula sus correspondiente parámetros. Este es descripto por la función **multiplicative_rayleigh_noise**.

Por otro lado, la distribución de Rayleigh que depende de un parámetro es diferente, ya que su distribución es la siguiente:

$f(x, \xi) = \frac{x}{\xi^2}e^{-x^2/(2\xi^2)}$

Siendo generada por:

$X = \xi \sqrt{-2 \, ln \, u)}$ con $u \sim U(0,1)$

Este último será descripto por **multiplicative_rayleigh_2**.

#### Rayleigh 1

In [ ]:
def multiplicative_rayleigh_noise(img, mean, var, dist_vals = None):
    b = (4 * var)/(4-np.pi)
    a = mean - np.sqrt(np.pi*b/4)
    print("a = %f , b = %f" % (a, b))
    IMG = np.copy(img)
    dist = lambda u: a + np.sqrt(-b*np.log(1-u))
    for i in range(len(IMG)):
        for j in range(len(IMG[0])):
            val = dist(random.uniform(0,1))
            if dist_vals is not None: dist_vals.append(val)
            IMG[i,j] = int(float(IMG[i,j]) * val)
    return IMG

In [ ]:
dist_rayleigh = []
noised_img = multiplicative_rayleigh_noise(img, 1, .02, dist_rayleigh)
plt.hist(dist_rayleigh, bins=100)
show_img_s([img ,noised_img])

#### Rayleigh 2

In [ ]:
def multiplicative_rayleigh_2(img, xi):
    IMG = np.copy(img)
    dist = lambda u: xi*np.sqrt(-2*np.log(u))
    for i in range(len(IMG)):
        for j in range(len(IMG[0])):
            val = dist(random.uniform(0,1))
            IMG[i,j] = int(float(IMG[i,j]) * val)
    return IMG

In [ ]:
xi_range = [.1,.5, 1, 2]
contamined_imgs = [multiplicative_rayleigh_2(img,x) for x in xi_range]
contamined_imgs_labels = ["XI = " + str(x) for x in xi_range]
print("------------ IMAGENES CONTAMINADAS ------------")
show_img_s(contamined_imgs, contamined_imgs_labels)
print("------------ FILTRO MEDIA ------------")
show_img_s([convolve(i, mean_kern(3)) for i in contamined_imgs])

In [ ]:
print("------------ UNSHARP_MASKSING ------------")
show_img_s([unsharp_masking(i, .7) for i in contamined_imgs])

## Ejercicio 5

Notar que no se lidió con los borden en la implementación del filtro. La decisión fue tomada debido que al hacerlo, se debería usar una estimación de los datos que faltan a considerar ahí, o reducir la dimensión de la ventana que se toma en cuenta en los casos borde. De hacerlo, si dicha ventana tuviera alta concentración de *outliers*, se deterioraría la calidad final.

In [ ]:
def median_filter(I, dim):
    m = len(I); n = len(I[0])
    out = np.zeros((m,n))
    for i in range(0+dim//2, m-dim//2):
        for j in range(0+dim//2, n-dim//2):
            t = I[i-dim//2:i+dim//2+1, j-dim//2:j+dim//2+1]
            out[i,j] = np.median(t)
    return out

In [ ]:
plt.figure(figsize=(18,10)); plt.axis('off')
plt.imshow(median_filter(img, 5), cmap='gray')
plt.show()

## Ejercicio 6

In [ ]:
p_range = [.01, .1, .2]
contamined_imgs = [pep_salt_noise(img,p) for p in p_range]
contamined_imgs_labels = ["p = " + str(p) for p in p_range]
print("------------ IMAGENES CONTAMINADAS ------------")
show_img_s(contamined_imgs, contamined_imgs_labels)
print("------------ FILTRO MEDIANA ------------")
show_img_s([median_filter(i, 3) for i in contamined_imgs])

## Ejercicio 7

In [ ]:
img = misc.imread('lena.png')

#### Roberts

In [ ]:
# Prewitt
Gx = [[-1,0,1],[-1,0,1],[-1,0,1]]
Gy = [[1,1,1],[0,0,0],[-1,-1,-1]]
img_x = convolve2d(img,Gx,mode='same')
img_y = convolve2d(img,Gy,mode='same')
img_mod = np.sqrt(np.power(img_x,2)+np.power(img_y,2))
show_img_s([img_x,img_y,img_mod])

#### Prewitt

In [ ]:
# Prewitt
Gx = [[-1,0,1],[-1,0,1],[-1,0,1]]
Gy = [[1,1,1],[0,0,0],[-1,-1,-1]]
img_x = convolve2d(img,Gx,mode='same')
img_y = convolve2d(img,Gy,mode='same')
img_mod = np.sqrt(np.power(img_x,2)+np.power(img_y,2))
show_img_s([img_x,img_y,img_mod])

#### Sobel

In [ ]:
# Sobel
Gx = [[-1,0,1],[-2,0,2],[-1,0,1]]
Gy = [[1,2,1],[0,0,0],[-1,-2,-1]]
img_x = convolve2d(img,Gx,mode='same')
img_y = convolve2d(img,Gy,mode='same')
img_mod = np.sqrt(np.power(img_x,2)+np.power(img_y,2))
show_img_s([img_x,img_y,img_mod])